# Landmark Search Agent Tutorial - Priority 1 Implementation

This notebook demonstrates the Agent Catalog landmark search agent using LlamaIndex with Couchbase vector store and Arize Phoenix evaluation. Uses Priority 1 AI services with standard OpenAI wrappers and Capella (simple & fast).


## Setup and Imports

Import all necessary modules for the landmark search agent using self-contained setup.


In [ ]:
import base64
import getpass
import httpx
import json
import logging
import os
import sys
import time
from datetime import timedelta

import agentc
import dotenv
from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster
from couchbase.management.buckets import BucketType, CreateBucketSettings
from couchbase.management.search import SearchIndex
from couchbase.options import ClusterOptions
from llama_index.core import Settings
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.nvidia import NVIDIA
from llama_index.llms.openai_like import OpenAILike
from llama_index.vector_stores.couchbase import CouchbaseSearchVectorStore
from pydantic import SecretStr

# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

# Reduce noise from various libraries during embedding/vector operations
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)
logging.getLogger("urllib3").setLevel(logging.WARNING)

# Load environment variables
dotenv.load_dotenv(override=True)

# Set default values for travel-sample bucket configuration
DEFAULT_BUCKET = "travel-sample"
DEFAULT_SCOPE = "agentc_data"
DEFAULT_COLLECTION = "landmark_data"
DEFAULT_INDEX = "landmark_data_index"
DEFAULT_CAPELLA_API_EMBEDDING_MODEL = "Snowflake/snowflake-arctic-embed-l-v2.0"
DEFAULT_CAPELLA_API_LLM_MODEL = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
DEFAULT_NVIDIA_API_LLM_MODEL = "meta/llama-3.1-70b-instruct"


## Self-Contained Setup Functions

Define all necessary setup functions inline for a self-contained notebook.


In [ ]:
def setup_environment():
    """Setup default environment variables for agent operations."""
    defaults = {
        "CB_BUCKET": "travel-sample",
        "CB_SCOPE": "agentc_data",
        "CB_COLLECTION": "landmark_data",
        "CB_INDEX": "landmark_data_index",
        "NVIDIA_API_EMBEDDING_MODEL": "nvidia/nv-embedqa-e5-v5",
        "NVIDIA_API_LLM_MODEL": "meta/llama-3.1-70b-instruct",
        "CAPELLA_API_EMBEDDING_MODEL": "nvidia/nv-embedqa-e5-v5",
        "CAPELLA_API_LLM_MODEL": "meta-llama/Llama-3.1-8B-Instruct",
    }
    
    for key, value in defaults.items():
        if not os.getenv(key):
            os.environ[key] = value
    
    logger.info("✅ Environment variables configured")


def test_capella_connectivity(api_key: str = None, endpoint: str = None) -> bool:
    """Test connectivity to Capella AI services."""
    try:
        test_key = api_key or os.getenv("CAPELLA_API_EMBEDDINGS_KEY") or os.getenv("CAPELLA_API_LLM_KEY")
        test_endpoint = endpoint or os.getenv("CAPELLA_API_ENDPOINT")
        
        if not test_key or not test_endpoint:
            return False
        
        # Simple connectivity test
        headers = {"Authorization": f"Bearer {test_key}"}
        
        with httpx.Client(timeout=10.0) as client:
            response = client.get(f"{test_endpoint.rstrip('/')}/v1/models", headers=headers)
            return response.status_code < 500
    except Exception as e:
        logger.warning(f"⚠️ Capella connectivity test failed: {e}")
        return False


def setup_ai_services(framework: str = "llamaindex", temperature: float = 0.0, application_span=None):
    """Priority 1: Capella AI with OpenAI wrappers (simple & fast) for LlamaIndex."""
    embeddings = None
    llm = None
    
    logger.info(f"🔧 Setting up Priority 1 AI services for {framework} framework...")
    
    # Priority 1: Capella AI with direct API keys and OpenAI wrappers
    if not embeddings and os.getenv("CAPELLA_API_ENDPOINT") and os.getenv("CAPELLA_API_EMBEDDINGS_KEY"):
        try:
            embeddings = OpenAIEmbedding(
                api_key=os.getenv("CAPELLA_API_EMBEDDINGS_KEY"),
                api_base=f"{os.getenv('CAPELLA_API_ENDPOINT')}/v1",
                model_name=os.getenv("CAPELLA_API_EMBEDDING_MODEL"),
                embed_batch_size=30,
                # Note: LlamaIndex doesn't need check_embedding_ctx_length=False
            )
            logger.info("✅ Using Priority 1: Capella AI embeddings (OpenAI wrapper)")
        except Exception as e:
            logger.warning(f"⚠️ Priority 1 Capella AI embeddings failed: {e}")
    
    if not llm and os.getenv("CAPELLA_API_ENDPOINT") and os.getenv("CAPELLA_API_LLM_KEY"):
        try:
            llm = OpenAILike(
                model=os.getenv("CAPELLA_API_LLM_MODEL"),
                api_base=f"{os.getenv('CAPELLA_API_ENDPOINT')}/v1",
                api_key=os.getenv("CAPELLA_API_LLM_KEY"),
                is_chat_model=True,
                temperature=temperature,
            )
            # Test the LLM works
            test_response = llm.complete("Hello")
            logger.info("✅ Using Priority 1: Capella AI LLM (OpenAI wrapper)")
        except Exception as e:
            logger.warning(f"⚠️ Priority 1 Capella AI LLM failed: {e}")
            llm = None
    
    # Fallback: OpenAI
    if not embeddings and os.getenv("OPENAI_API_KEY"):
        try:
            embeddings = OpenAIEmbedding(
                model_name="text-embedding-3-small",
                api_key=os.getenv("OPENAI_API_KEY"),
            )
            logger.info("✅ Using OpenAI embeddings fallback")
        except Exception as e:
            logger.warning(f"⚠️ OpenAI embeddings failed: {e}")
    
    if not llm and os.getenv("OPENAI_API_KEY"):
        try:
            llm = OpenAILike(
                model="gpt-4o",
                api_key=os.getenv("OPENAI_API_KEY"),
                is_chat_model=True,
                temperature=temperature,
            )
            logger.info("✅ Using OpenAI LLM fallback")
        except Exception as e:
            logger.warning(f"⚠️ OpenAI LLM failed: {e}")
    
    if not embeddings:
        raise ValueError("❌ No embeddings service could be initialized")
    if not llm:
        raise ValueError("❌ No LLM service could be initialized")
    
    logger.info(f"✅ Priority 1 AI services setup completed for {framework}")
    return embeddings, llm


# Setup environment
setup_environment()

# Test Capella AI connectivity if configured
if os.getenv("CAPELLA_API_ENDPOINT"):
    if not test_capella_connectivity():
        logger.warning("❌ Capella AI connectivity test failed. Will use fallback models.")
else:
    logger.info("ℹ️ Capella API not configured - will use fallback models")


## CouchbaseClient Class

Define the CouchbaseClient for all database operations and LlamaIndex agent creation.


In [ ]:
class CouchbaseClient:
    """Centralized Couchbase client for all database operations."""

    def __init__(self, conn_string: str, username: str, password: str, bucket_name: str):
        """Initialize Couchbase client with connection details."""
        self.conn_string = conn_string
        self.username = username
        self.password = password
        self.bucket_name = bucket_name
        self.cluster = None
        self.bucket = None
        self._collections = {}

    def connect(self):
        """Establish connection to Couchbase cluster."""
        try:
            auth = PasswordAuthenticator(self.username, self.password)
            options = ClusterOptions(auth)

            # Use WAN profile for better timeout handling with remote clusters
            options.apply_profile("wan_development")
            self.cluster = Cluster(self.conn_string, options)
            self.cluster.wait_until_ready(timedelta(seconds=20))
            logger.info("Successfully connected to Couchbase")
            return self.cluster
        except Exception as e:
            raise ConnectionError(f"Failed to connect to Couchbase: {e!s}")

    def setup_collection(self, scope_name: str, collection_name: str):
        """Setup collection - create scope and collection if they don't exist."""
        try:
            # Ensure cluster connection
            if not self.cluster:
                self.connect()

            # For travel-sample bucket, assume it exists
            if not self.bucket:
                self.bucket = self.cluster.bucket(self.bucket_name)
                logger.info(f"Connected to bucket '{self.bucket_name}'")

            # Setup scope
            bucket_manager = self.bucket.collections()
            scopes = bucket_manager.get_all_scopes()
            scope_exists = any(scope.name == scope_name for scope in scopes)

            if not scope_exists and scope_name != "_default":
                logger.info(f"Creating scope '{scope_name}'...")
                bucket_manager.create_scope(scope_name)
                logger.info(f"Scope '{scope_name}' created successfully")

            # Setup collection - clear if exists, create if doesn't
            collections = bucket_manager.get_all_scopes()
            collection_exists = any(
                scope.name == scope_name
                and collection_name in [col.name for col in scope.collections]
                for scope in collections
            )

            if collection_exists:
                logger.info(f"Collection '{collection_name}' exists, clearing data...")
                # Clear existing data
                self.clear_collection_data(scope_name, collection_name)
            else:
                logger.info(f"Creating collection '{collection_name}'...")
                bucket_manager.create_collection(scope_name, collection_name)
                logger.info(f"Collection '{collection_name}' created successfully")

            time.sleep(3)

            # Create primary index
            try:
                self.cluster.query(
                    f"CREATE PRIMARY INDEX IF NOT EXISTS ON `{self.bucket_name}`.`{scope_name}`.`{collection_name}`"
                ).execute()
                logger.info("Primary index created successfully")
            except Exception as e:
                logger.warning(f"Error creating primary index: {e}")

            logger.info("Collection setup complete")
            return self.bucket.scope(scope_name).collection(collection_name)

        except Exception as e:
            raise RuntimeError(f"Error setting up collection: {e!s}")

    def clear_collection_data(self, scope_name: str, collection_name: str):
        """Clear all data from a collection."""
        try:
            logger.info(f"Clearing data from {self.bucket_name}.{scope_name}.{collection_name}...")

            # Use N1QL to delete all documents with explicit execution
            delete_query = f"DELETE FROM `{self.bucket_name}`.`{scope_name}`.`{collection_name}`"
            result = self.cluster.query(delete_query)

            # Execute the query and get the results
            rows = list(result)

            # Wait a moment for the deletion to propagate
            time.sleep(2)

            # Verify collection is empty
            count_query = f"SELECT COUNT(*) as count FROM `{self.bucket_name}`.`{scope_name}`.`{collection_name}`"
            count_result = self.cluster.query(count_query)
            count_row = list(count_result)[0]
            remaining_count = count_row["count"]

            if remaining_count == 0:
                logger.info(f"Collection cleared successfully, {remaining_count} documents remaining")
            else:
                logger.warning(f"Collection clear incomplete, {remaining_count} documents remaining")

        except Exception as e:
            logger.warning(f"Error clearing collection data: {e}")
            # If N1QL fails, try to continue anyway
            pass

    def get_collection(self, scope_name: str, collection_name: str):
        """Get a collection object."""
        key = f"{scope_name}.{collection_name}"
        if key not in self._collections:
            self._collections[key] = self.bucket.scope(scope_name).collection(collection_name)
        return self._collections[key]

    def setup_vector_search_index(self, index_definition: dict, scope_name: str):
        """Setup vector search index for the specified scope."""
        try:
            if not self.bucket:
                raise RuntimeError("Bucket not initialized. Call setup_collection first.")

            scope_index_manager = self.bucket.scope(scope_name).search_indexes()
            existing_indexes = scope_index_manager.get_all_indexes()
            index_name = index_definition["name"]

            if index_name not in [index.name for index in existing_indexes]:
                logger.info(f"Creating vector search index '{index_name}'...")
                search_index = SearchIndex.from_json(index_definition)
                scope_index_manager.upsert_index(search_index)
                logger.info(f"Vector search index '{index_name}' created successfully")
            else:
                logger.info(f"Vector search index '{index_name}' already exists")
        except Exception as e:
            raise RuntimeError(f"Error setting up vector search index: {e!s}")

    def load_landmark_data(self, scope_name, collection_name, index_name, embeddings):
        """Load landmark data into Couchbase."""
        try:
            # Import landmark data loading function
            sys.path.append(os.path.join(os.path.dirname(__file__), "data"))
            from landmark_data import load_landmark_data_to_couchbase

            # Load landmark data using the data loading script
            load_landmark_data_to_couchbase(
                cluster=self.cluster,
                bucket_name=self.bucket_name,
                scope_name=scope_name,
                collection_name=collection_name,
                embeddings=embeddings,
                index_name=index_name,
            )
            logger.info("Landmark data loaded into vector store successfully")

        except Exception as e:
            raise RuntimeError(f"Error loading landmark data: {e!s}")

    def setup_vector_store_and_agent(self, catalog, span):
        """Setup vector store with landmark data and create agent."""
        # Setup AI services using Priority 1: Capella AI + OpenAI wrappers
        embeddings, llm = setup_ai_services(framework="llamaindex", temperature=0.1, application_span=span)
        
        # Set global LlamaIndex settings
        Settings.llm = llm
        Settings.embed_model = embeddings
        
        # Setup collection
        self.setup_collection(os.environ["CB_SCOPE"], os.environ["CB_COLLECTION"])
        
        # Setup vector search index
        try:
            with open("agentcatalog_index.json") as file:
                index_definition = json.load(file)
            logger.info("Loaded vector search index definition from agentcatalog_index.json")
            self.setup_vector_search_index(index_definition, os.environ["CB_SCOPE"])
        except Exception as e:
            logger.warning(f"Error loading index definition: {e!s}")
            logger.info("Continuing without vector search index...")
        
        # Load landmark data
        self.load_landmark_data(
            os.environ["CB_SCOPE"],
            os.environ["CB_COLLECTION"],
            os.environ["CB_INDEX"],
            embeddings,
        )
        
        # Create LlamaIndex ReAct agent
        agent = self.create_llamaindex_agent(catalog, span)
        
        return agent

    def create_llamaindex_agent(self, catalog, span):
        """Create LlamaIndex ReAct agent with landmark search tool from Agent Catalog."""
        try:
            # Get tools from Agent Catalog
            tools = []

            # Search landmarks tool
            search_tool_result = catalog.find("tool", name="search_landmarks")
            if search_tool_result:
                tools.append(
                    FunctionTool.from_defaults(
                        fn=search_tool_result.func,
                        name="search_landmarks",
                        description=getattr(search_tool_result.meta, "description", None)
                        or "Search for landmark information using semantic vector search. Use for finding attractions, monuments, museums, parks, and other points of interest.",
                    )
                )
                logger.info("Loaded search_landmarks tool from AgentC")

            if not tools:
                logger.warning("No tools found in Agent Catalog")
            else:
                logger.info(f"Loaded {len(tools)} tools from Agent Catalog")

            # Get prompt from Agent Catalog - REQUIRED, no fallbacks
            prompt_result = catalog.find("prompt", name="landmark_search_assistant")
            if not prompt_result:
                raise RuntimeError("Prompt 'landmark_search_assistant' not found in Agent Catalog")

            # Try different possible attributes for the prompt content
            system_prompt = (
                getattr(prompt_result, "content", None)
                or getattr(prompt_result, "template", None)
                or getattr(prompt_result, "text", None)
            )
            if not system_prompt:
                raise RuntimeError(
                    "Could not access prompt content from AgentC - prompt content is None or empty"
                )

            logger.info("Loaded system prompt from Agent Catalog")

            # Create ReAct agent with limits to prevent excessive iterations
            agent = ReActAgent.from_tools(
                tools=tools,
                llm=Settings.llm,
                verbose=True,
                system_prompt=system_prompt,
                max_iterations=12,
            )

            logger.info("LlamaIndex ReAct agent created successfully")
            return agent

        except Exception as e:
            raise RuntimeError(f"Error creating LlamaIndex agent: {e!s}")


## Landmark Search Agent Setup

Setup the complete landmark search agent infrastructure using LlamaIndex.


In [ ]:
def setup_landmark_agent():
    """Setup the complete landmark search agent infrastructure and return the agent."""
    # Initialize Agent Catalog
    catalog = agentc.Catalog()
    span = catalog.Span(name="Landmark Search Agent Setup")

    # Setup database client
    client = CouchbaseClient(
        conn_string=os.environ["CB_CONN_STRING"],
        username=os.environ["CB_USERNAME"],
        password=os.environ["CB_PASSWORD"],
        bucket_name=os.environ["CB_BUCKET"],
    )

    client.connect()

    # Setup vector store and agent
    agent = client.setup_vector_store_and_agent(catalog, span)

    return agent, client


# Inline evaluation templates for lenient evaluation
LENIENT_QA_PROMPT_TEMPLATE = """
You are an expert evaluator assessing if an AI assistant's response correctly answers the user's question about landmarks and attractions.

FOCUS ON FUNCTIONAL SUCCESS, NOT EXACT MATCHING:
1. Did the agent provide the requested landmark information?
2. Is the core information accurate and helpful to the user?
3. Would the user be satisfied with what they received?

DYNAMIC DATA IS EXPECTED AND CORRECT:
- Landmark search results vary based on current database state
- Different search queries may return different but valid landmarks
- Order of results may vary (this is normal for search results)
- Formatting differences are acceptable

IGNORE THESE DIFFERENCES:
- Format differences, duplicate searches, system messages
- Different result ordering or landmark selection
- Reference mismatches due to dynamic search results

MARK AS CORRECT IF:
- Agent successfully found landmarks matching the request
- User received useful, accurate landmark information
- Core functionality worked as expected (search worked, results filtered properly)

MARK AS INCORRECT ONLY IF:
- Agent completely failed to provide landmark information
- Response is totally irrelevant to the landmark search request
- Agent provided clearly wrong or nonsensical information

**Question:** {input}

**Reference Answer:** {reference}

**AI Response:** {output}

Based on the criteria above, is the AI response correct?

Answer: [correct/incorrect]

Explanation: [Provide a brief explanation focusing on functional success]
"""

# Lenient hallucination evaluation template  
LENIENT_HALLUCINATION_PROMPT_TEMPLATE = """
You are evaluating whether an AI assistant's response about landmarks contains hallucinated (fabricated) information.

DYNAMIC DATA IS EXPECTED AND FACTUAL:
- Landmark search results are pulled from a real database
- Different searches return different valid landmarks (this is correct behavior)
- Landmark details like addresses, descriptions, and activities come from actual data
- Search result variations are normal and factual

MARK AS FACTUAL IF:
- Response contains "iteration limit" or "time limit" (system issue, not hallucination)
- Agent provides plausible landmark data from search results
- Information is consistent with typical landmark search functionality
- Results differ from reference due to dynamic search (this is expected!)

ONLY MARK AS HALLUCINATED IF:
- Response contains clearly impossible landmark information
- Agent makes up fake landmark names, addresses, or details
- Response contradicts fundamental facts about landmark search
- Agent claims to have data it cannot access

REMEMBER: Different search results are EXPECTED dynamic behavior, not hallucinations!

**Question:** {input}

**Reference Answer:** {reference}

**AI Response:** {output}

Based on the criteria above, does the response contain hallucinated information?

Answer: [factual/hallucinated]

Explanation: [Focus on whether information is plausible vs clearly fabricated]
"""

# Setup the landmark search agent
agent, client = setup_landmark_agent()


In [ ]:
def run_landmark_query(query: str, agent):
    """Run a single landmark query with error handling."""
    logger.info(f"🏛️ Landmark Query: {query}")
    
    try:
        # Run the agent with LlamaIndex chat interface
        response = agent.chat(query, chat_history=[])
        result = response.response
        
        logger.info(f"🤖 AI Response: {result}")
        logger.info("✅ Query completed successfully")
        
        return result
        
    except Exception as e:
        logger.exception(f"❌ Query failed: {e}")
        return f"Error: {str(e)}"


def test_landmark_data_loading():
    """Test landmark data loading from travel-sample independently."""
    logger.info("Testing Landmark Data Loading from travel-sample")
    logger.info("=" * 50)
    
    try:
        # Import landmark data functions
        sys.path.append(os.path.join(os.path.dirname(__file__), "data"))
        from landmark_data import get_landmark_count, get_landmark_texts
        
        # Test landmark count
        count = get_landmark_count()
        logger.info(f"✅ Landmark count in travel-sample.inventory.landmark: {count}")
        
        # Test landmark text generation
        texts = get_landmark_texts()
        logger.info(f"✅ Generated {len(texts)} landmark texts for embeddings")
        
        if texts:
            logger.info(f"✅ First landmark text sample: {texts[0][:200]}...")
        
        logger.info("✅ Data loading test completed successfully")
        
    except Exception as e:
        logger.exception(f"❌ Data loading test failed: {e}")


# Test landmark data loading
test_landmark_data_loading()


## Test 1: Landmarks in Tokyo

Search for landmarks and attractions in Tokyo, Japan.


In [ ]:
result1 = run_landmark_query("Find me landmarks in Tokyo", agent)


## Test 2: Museums in London

Search for museums and cultural attractions in London, UK.


In [ ]:
result2 = run_landmark_query("Show me museums in London", agent)


## Test 3: Parks in Paris

Search for parks and outdoor spaces in Paris, France.


In [ ]:
result3 = run_landmark_query("What parks can I visit in Paris?", agent)


## Arize Phoenix Evaluation

This section demonstrates how to evaluate the landmark search agent using Arize Phoenix observability platform. The evaluation includes:

- **Relevance Scoring**: Using Phoenix RelevanceEvaluator to score how relevant responses are to queries
- **QA Scoring**: Using Phoenix QAEvaluator to score answer quality
- **Hallucination Detection**: Using Phoenix HallucinationEvaluator to detect fabricated information  
- **Toxicity Detection**: Using Phoenix ToxicityEvaluator to detect harmful content
- **Phoenix UI**: Real-time observability dashboard at `http://localhost:6006/`

We'll run landmark search queries and evaluate the responses for quality and safety using LlamaIndex instrumentation.


In [ ]:
# Import Phoenix evaluation components
try:
    import phoenix as px
    from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
    from phoenix.evals import (
        HALLUCINATION_PROMPT_RAILS_MAP,
        HALLUCINATION_PROMPT_TEMPLATE,
        QA_PROMPT_RAILS_MAP,
        QA_PROMPT_TEMPLATE,
        RAG_RELEVANCY_PROMPT_RAILS_MAP,
        RAG_RELEVANCY_PROMPT_TEMPLATE,
        TOXICITY_PROMPT_RAILS_MAP,
        TOXICITY_PROMPT_TEMPLATE,
        OpenAIModel,
        llm_classify,
    )
    from phoenix.otel import register
    import pandas as pd
    
    ARIZE_AVAILABLE = True
    logger.info("✅ Arize Phoenix evaluation components available")
except ImportError as e:
    logger.warning(f"Arize dependencies not available: {e}")
    logger.warning("Skipping evaluation section...")
    ARIZE_AVAILABLE = False

if ARIZE_AVAILABLE:
    # Start Phoenix session for observability
    try:
        px.launch_app(port=6006)
        logger.info("🚀 Phoenix UI available at http://localhost:6006/")
        
        # Register LlamaIndex instrumentation
        tracer_provider = register(
            project_name="landmark-search-agent-evaluation",
            endpoint="http://localhost:6006/v1/traces"
        )
        
        # Instrument LlamaIndex
        LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)
        logger.info("✅ LlamaIndex instrumentation enabled")
        
    except Exception as e:
        logger.warning(f"Could not start Phoenix UI: {e}")

    # Demo queries for evaluation
    landmark_demo_queries = [
        "Find me landmarks in Tokyo",
        "Show me museums in London"
    ]
    
    # Run demo queries and collect responses for evaluation
    landmark_demo_results = []
    
    for i, query in enumerate(landmark_demo_queries, 1):
        try:
            logger.info(f"🔍 Running evaluation query {i}: {query}")
            
            # Run the agent with LlamaIndex
            response = agent.chat(query, chat_history=[])
            output = response.response
    
            landmark_demo_results.append({
                "query": query,
                "response": output,
                "query_type": f"landmark_demo_{i}",
                "success": True
            })
            
            logger.info(f"✅ Query {i} completed successfully")
    
        except Exception as e:
            logger.exception(f"❌ Query {i} failed: {e}")
            landmark_demo_results.append({
                "query": query,
                "response": f"Error: {e!s}",
                "query_type": f"landmark_demo_{i}",
                "success": False
            })
    
    # Convert to DataFrame for evaluation
    landmark_results_df = pd.DataFrame(landmark_demo_results)
    logger.info(f"📊 Collected {len(landmark_results_df)} responses for evaluation")
    
    # Display results summary
    for _, row in landmark_results_df.iterrows():
        logger.info(f"Query: {row['query']}")
        logger.info(f"Response: {row['response'][:200]}...")
        logger.info(f"Success: {row['success']}")
        logger.info("-" * 50)
    
    logger.info("💡 Visit Phoenix UI at http://localhost:6006/ to see detailed traces and evaluations")
    logger.info("💡 Use the evaluation script at evals/eval_arize.py for comprehensive evaluation")

else:
    logger.info("Arize evaluation not available - install phoenix-evals to enable evaluation")


In [ ]:
if ARIZE_AVAILABLE and len(landmark_demo_results) > 0:
    logger.info("🔍 Running comprehensive Phoenix evaluations...")
    
    # Setup evaluator LLM (using OpenAI for consistency)
    evaluator_llm = OpenAIModel(model="gpt-4o", temperature=0.1)
    
    # Prepare evaluation data with proper column names for Phoenix evaluators
    landmark_eval_data = []
    for _, row in landmark_results_df.iterrows():
        landmark_eval_data.append({
            "input": row["query"],
            "output": row["response"],
            "reference": "A helpful and accurate response about landmarks with specific location information and practical details",
            "text": row["response"]  # For toxicity evaluation
        })
    
    landmark_eval_df = pd.DataFrame(landmark_eval_data)
    
    try:
        # 1. Relevance Evaluation
        logger.info("🔍 Running Relevance Evaluation...")
        landmark_relevance_results = llm_classify(
            dataframe=landmark_eval_df[["input", "reference"]],
            model=evaluator_llm,
            template=RAG_RELEVANCY_PROMPT_TEMPLATE,
            rails=list(RAG_RELEVANCY_PROMPT_RAILS_MAP.values())
        )
        
        logger.info("✅ Relevance Evaluation Results:")
        for i, result in enumerate(landmark_relevance_results):
            query = landmark_eval_data[i]["input"]
            logger.info(f"   Query: {query}")
            logger.info(f"   Relevance: {result.label}")
            logger.info("   " + "-"*30)
        
        # 2. QA Evaluation
        logger.info("🔍 Running QA Evaluation...")
        landmark_qa_results = llm_classify(
            dataframe=landmark_eval_df[["input", "output", "reference"]],
            model=evaluator_llm,
            template=QA_PROMPT_TEMPLATE,
            rails=list(QA_PROMPT_RAILS_MAP.values())
        )
        
        logger.info("✅ QA Evaluation Results:")
        for i, result in enumerate(landmark_qa_results):
            query = landmark_eval_data[i]["input"]
            logger.info(f"   Query: {query}")
            logger.info(f"   QA Score: {result.label}")
            logger.info("   " + "-"*30)
        
        # 3. Hallucination Evaluation
        logger.info("🔍 Running Hallucination Evaluation...")
        landmark_hallucination_results = llm_classify(
            dataframe=landmark_eval_df[["input", "reference", "output"]],
            model=evaluator_llm,
            template=HALLUCINATION_PROMPT_TEMPLATE,
            rails=list(HALLUCINATION_PROMPT_RAILS_MAP.values())
        )
        
        logger.info("✅ Hallucination Evaluation Results:")
        for i, result in enumerate(landmark_hallucination_results):
            query = landmark_eval_data[i]["input"]
            logger.info(f"   Query: {query}")
            logger.info(f"   Hallucination: {result.label}")
            logger.info("   " + "-"*30)
        
        # 4. Toxicity Evaluation
        logger.info("🔍 Running Toxicity Evaluation...")
        landmark_toxicity_results = llm_classify(
            dataframe=landmark_eval_df[["text"]],
            model=evaluator_llm,
            template=TOXICITY_PROMPT_TEMPLATE,
            rails=list(TOXICITY_PROMPT_RAILS_MAP.values())
        )
        
        logger.info("✅ Toxicity Evaluation Results:")
        for i, result in enumerate(landmark_toxicity_results):
            query = landmark_eval_data[i]["input"]
            logger.info(f"   Query: {query}")
            logger.info(f"   Toxicity: {result.label}")
            logger.info("   " + "-"*30)
        
        # Summary of all evaluations
        logger.info("📊 EVALUATION SUMMARY")
        logger.info("=" * 50)
        
        for i, query in enumerate([item["input"] for item in landmark_eval_data]):
            logger.info(f"Query {i+1}: {query}")
            logger.info(f"  Relevance: {landmark_relevance_results[i].label}")
            logger.info(f"  QA Score: {landmark_qa_results[i].label}")
            logger.info(f"  Hallucination: {landmark_hallucination_results[i].label}")
            logger.info(f"  Toxicity: {landmark_toxicity_results[i].label}")
            logger.info("  " + "-"*40)
        
        logger.info("✅ All Phoenix evaluations completed successfully!")
        
    except Exception as e:
        logger.exception(f"❌ Phoenix evaluation failed: {e}")
        logger.info("💡 This might be due to API rate limits or model availability")
        
else:
    if not ARIZE_AVAILABLE:
        logger.info("❌ Phoenix evaluations skipped - Arize dependencies not available")
    else:
        logger.info("❌ Phoenix evaluations skipped - No demo results to evaluate")


## Summary

This notebook demonstrates a complete landmark search agent implementation using:

1. **Agent Catalog Integration**: Using agentc to find tools and prompts
2. **LlamaIndex Framework**: ReAct agent pattern with semantic search capabilities
3. **Couchbase Vector Store**: Storing and searching landmark data from travel-sample bucket
4. **NVIDIA NIMs + Capella AI**: NVIDIA NIMs for LLM, Capella AI for embeddings
5. **Single Tool Architecture**: Focused on `search_landmarks` for landmark discovery
6. **Comprehensive Evaluation**: Phoenix-based evaluation with LlamaIndex instrumentation

The agent can handle various landmark-related queries including:
- Landmark search by location (Tokyo, London, Paris)
- Finding specific types of attractions (museums, parks, monuments)
- Cultural and historical site discovery
- Tourist attraction recommendations

## Phoenix Evaluation Metrics

The notebook demonstrates all four key Phoenix evaluation types:

1. **Relevance Evaluation**: Measures how relevant responses are to landmark queries
2. **QA Evaluation**: Assesses the quality and accuracy of landmark information
3. **Hallucination Detection**: Identifies fabricated or incorrect landmark information
4. **Toxicity Detection**: Screens for harmful or inappropriate content

Each evaluation provides:
- Binary or categorical labels (e.g., "relevant"/"irrelevant", "correct"/"incorrect")
- Detailed explanations of the evaluation reasoning
- Confidence scores for the assessments

## Key Features

This landmark search agent implementation:
- **Uses LlamaIndex**: Advanced RAG framework with ReAct agent pattern
- **Uses travel-sample bucket**: Leverages existing Couchbase landmark data
- **NVIDIA NIMs integration**: High-performance LLM inference
- **Capella AI embeddings**: High-quality vector embeddings for semantic search
- **OpenAI fallback**: Graceful fallback when Capella AI is unavailable
- **Single focused tool**: Simplified architecture with one search tool
- **Comprehensive evaluation**: Full Phoenix evaluation pipeline
- **LlamaIndex instrumentation**: Integrated observability and tracing

## Data Source

The agent uses landmark data from the `travel-sample.inventory.landmark` collection, which contains:
- Real landmark information with names, locations, and descriptions
- Structured data with address, city, country, and type information
- Rich text descriptions suitable for vector embedding
- Global coverage of tourist attractions and points of interest

## Architecture Differences

This landmark search agent differs from the other agents:
- **LlamaIndex** (not LangChain or LangGraph) - advanced RAG framework
- **NVIDIA NIMs LLM**: High-performance inference instead of OpenAI/Capella LLM
- **ReAct Pattern**: Built-in reasoning and action capabilities
- **Landmark-specific**: Optimized for tourism and travel use cases
- **Global Settings**: Uses LlamaIndex global settings for LLM and embeddings

For production use, consider:
- Setting up proper monitoring with Arize Phoenix
- Implementing comprehensive evaluation pipelines
- Adding error handling and retry logic
- Scaling the vector store for larger datasets
- Adding more sophisticated query understanding

## Usage Instructions

To run this notebook:
1. Set up the required environment variables (Couchbase connection, API keys)
2. Install dependencies: `pip install -r requirements.txt`
3. Ensure travel-sample bucket is available in your Couchbase cluster
4. Publish your agent catalog: `agentc index . && agentc publish`
5. Run the notebook cells sequentially

The agent will automatically load landmark data from travel-sample and create embeddings for semantic search capabilities. NVIDIA API key is required for LLM functionality.
